In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

import torch

from scipy.stats import multivariate_normal

from utils import *

# Lecture 2.1: Markov processes

* Intro to Markov Chain Monte Carlo
* Markov processes, stationary distribution and ergodicity

MacKay, **Chapter 29**.

# Markov Chain Monte Carlo: the general idea with three examples

#### Example of a Markov process in 2 dimension: a refresher on diffusion

In [ ]:
eps = 0.5
num_steps = 5
xlim = 4
np.random.seed(12) # choose a seed (this is not an appropriate seed)

# start from the origin and generate the next point using an isotropic Gaussian step in 2 dimensions
_, _, xgrid_stacked = get_wgrid(wlim=xlim, num=100)
xs = np.zeros((num_steps, 2))
x = np.zeros(2) # start at zero
fig, axes = plt.subplots(num_steps, figsize=(12,2 * num_steps))
for t in range(num_steps):
    pgrid = multivariate_normal(x, np.eye(2) * eps**2).pdf(xgrid_stacked)
    xs[t] = x
    x += np.random.randn(2) * eps # add a random gaussian at each time-step
    handle = axes[t]
    handle.set_xlim([-xlim,xlim])
    handle.set_ylim([-xlim,xlim])
    handle.set_aspect("equal")
    handle.plot(xs[:,0], xs[:,1], '.-', lw=0.5, c='black')
    handle.imshow(pgrid, origin="lower", extent=[-xlim,xlim,-xlim,xlim], alpha=0.6)

plt.tight_layout();

In [ ]:
# generate a bunch of random walks in D dimension and check the sqrt scaling for distances
D = 2
eps = 1e-1
samples = 100
num_steps = 1000

np.random.seed(42) # choose a seed (this is an appropriate seed [1])

X = np.cumsum(np.random.randn(samples, D, num_steps), -1) * eps # a faster way to generate multiple trajectories at once

plt.figure(figsize=(10,4))

plt.subplot(121)
sample = 1
plt.plot(X[sample,0,0], X[sample,1,0], 'o', ms=4, color="black", label="start");
plt.plot(X[sample,0,:], X[sample,1,:], alpha=0.7, color=color_cycle[0]);
plt.plot(X[sample,0,-1], X[sample,1,-1], 'x', ms=6, color="black", label="end");
plt.plot([X[sample,0,0], X[sample,0,-1]], [X[sample,1,0], X[sample,1,-1]], '--', color="gray", alpha=1);
plt.legend();

plt.subplot(122)
plt.plot((X[:10]**2).sum(1).T, alpha=0.3);
plt.plot((X**2).sum(1).mean(0), c='black', label="average")
plt.plot(np.arange(num_steps), eps**2 * D * np.arange(num_steps), '--', c='black', label=r"$d^2 \propto t$")
plt.xscale('log')
plt.yscale('log')
plt.legend();

plt.tight_layout();

In [ ]:
# check the marginal distribution over time
i = 0
for it, t in enumerate(np.linspace(20, num_steps-1, 5)):
    t = int(t)
    plt.hist(X[:,i,t], bins=50, density=True, alpha=0.5, color=color_cycle[it]);
    xs = np.linspace(-5*eps*np.sqrt(t),5*eps*np.sqrt(t), 100)
    plt.plot(xs, np.exp(-0.5/(eps**2*t)*xs**2)/np.sqrt(2*np.pi*eps**2*t), c=color_cycle[it], label=f"t = {t}");
plt.legend(); 

#### A Markov process converging to a prescribed distribution #1: the Ornstein-Uhlenbeck process

In [ ]:
# let's run an OU process starting from a delta initial condition
sigma = 0.1
k = 0.7
dt = 0.1
num_samples = 5000
num_steps = 500

# run a bunch of trajectories in parallel
x0 = 0.5
xs = np.zeros((num_steps, num_samples))
x = np.ones(num_samples) * x0
for t in range(num_steps):
    xs[t] = x
    x = (1 - dt * k) * x + np.sqrt(dt) * sigma * np.random.randn(num_samples)

# plot the initial steps of some trajectories
plt.plot(xs[:200,:10], '.-', alpha=0.1, c='black');

In [ ]:
ts = np.arange(num_steps) * dt
expkts = np.exp(-k * ts)
mean = x0 * expkts
var = 0.5*sigma**2/k * (1-expkts**2)

plt.figure(figsize=(10,4))

plt.subplot(121)
plt.plot(xs.mean(-1), label='av data')
plt.plot(mean, label='av theory')
plt.legend();
plt.xscale('log')

plt.subplot(122)
plt.plot(xs.var(-1), label='var data')
plt.plot(var, label='var theory')
plt.legend();
plt.xscale('log')

plt.tight_layout();

In [ ]:
# visualize evolution of probability over states
num_to_show = 10
t_to_show = np.logspace(0, np.log10(num_steps), num_to_show, dtype=int) - 1

fig, axes = plt.subplots(num_to_show, figsize=(4, 2 * num_to_show))
x_to_plot = np.linspace(-1, 1, 1000)
for i, t in enumerate(t_to_show):
    axes[i].hist(xs[t], bins=100, density=True, alpha=0.5, label='data');
    axes[i].plot(x_to_plot, multivariate_normal(mean[t], var[t]+1e-5).pdf(x_to_plot), label='theory');
    axes[i].set_xlim([-1, 1])
    axes[i].set_title(f't = {t}')
    axes[i].set_xlabel('x')
    axes[i].set_ylabel('p(x)');

plt.tight_layout();

In [ ]:
# run it again from a more complicate distribution of initial conditions
sigma = 0.1
k = 0.7
dt = 0.1
num_samples = 5000
num_steps = 500

# run a bunch of trajectories in parallel
xs = np.zeros((num_steps, num_samples))

x = 0.05 * np.random.randn(num_samples) + 2 * np.concatenate([-np.ones(num_samples//2), np.ones(num_samples//2)])
for t in range(num_steps):
    xs[t] = x
    x = (1 - dt * k) * x + np.sqrt(dt) * sigma * np.random.randn(num_samples)

# plot the initial steps of some trajectories
plt.plot(xs[:100,:10], '.-', alpha=0.1, c='black');
plt.plot(xs[:100:,-10:], '.-', alpha=0.1, c='black');

In [ ]:
# visualize evolution of probability over states
num_to_show = 10
t_to_show = np.logspace(0, np.log10(num_steps), num_to_show, dtype=int) - 1

fig, axes = plt.subplots(num_to_show, figsize=(4, 2 * num_to_show))
x_to_plot = np.linspace(-3, 3, 100)
for i, t in enumerate(t_to_show):
    axes[i].hist(xs[t], bins=100, density=True, alpha=0.5, label='data');
    if t == t_to_show[-1]:
        axes[i].plot(x_to_plot, multivariate_normal(mean[t], var[t]+1e-5).pdf(x_to_plot), label='theory');
    axes[i].set_xlim([-3, 3])
    axes[i].set_title(f't = {t}')
    axes[i].set_xlabel('x')
    axes[i].set_ylabel('p(x)');

plt.tight_layout();

#### A Markov process converging to a prescribed distribution #2: the Metropolis-Hastings transition

In [ ]:
sigma = 1
eps = 0.5
xlim = 5
_, _, xgrid_stacked = get_wgrid(wlim=xlim, num=200)
distrib_to_sample = multivariate_normal(np.zeros(2), np.eye(2) * sigma**2)
pgrid_to_sample = distrib_to_sample.pdf(xgrid_stacked)

plt.figure(figsize=(10,4))

plt.subplot(121)
plt.imshow(pgrid_to_sample, extent=[-xlim,xlim,-xlim,xlim], alpha=0.6, origin="lower");
plt.xlabel('x')
plt.xlabel('y')
plt.title("p(x)")

plt.subplot(122)
x = np.array([3,-1])
# x = np.array([0,3]) # try some other points to see how transition points towards typical regions of the distribution
distrib_proposal = multivariate_normal(x, np.eye(2) * eps**2)
pgrid_proposal = distrib_proposal.pdf(xgrid_stacked)

q = np.minimum(np.ones_like(pgrid_to_sample), pgrid_to_sample/distrib_to_sample.pdf(x)) * pgrid_proposal
plt.plot(x[0], x[1], '.', c='black');
plt.imshow(q, extent=[-xlim,xlim,-xlim,xlim], alpha=0.6, origin="lower");
plt.xlabel('x')
plt.xlabel('y')
plt.title('MH transition')

plt.tight_layout();

# Markov processes: a bit of theory

### Definition

Let $\mathcal{S}$ denote the (finite) set of all state vectors $x$ [as an example, $x \in \mathcal{S}$ is a binary vector of length $n$ and thus $x$ can take on $2^n$ different values].

Denote $p(x)$ a probability distribution over states [in our example, $p(x)$ is a vector of length $2^n$ with $\sum_x p(x)=1$].

In our context, a **Markov process** is a *discrete-time* stochastic dynamical process that is defined by a transition matrix $T(x'|x)$ that specifies the transition probability from an initial state $x$ to a final state $x'$ in a single time step [in the example, $T(x'|x)$ is a $2^n \times 2^n$ matrix].

Note that for any $x$, $T(x'|x)$ is a probability vector in $x'$:
$$\sum_{x'} T(x'|x)=1$$
Matrices with this property (non-negative entries and normalized columns) are called **(left/column) stochastic matrices**.

The Markov dynamics is given by
$$p_{t+1}(x')=\sum_x T(x'|x)p_t(x)$$

where we assume we start from an initial state distribution $p_0(x)$. The probability update is thus a left-multiplication (right-multiplication would equivalently work with a right/row stochastic matrix).

### Stationarity

In the setting of a MCMC, we will build a Markov process
$$p_0 \to p_1 = T p_0 \to p_2 = Tp_1 \to p_n=T^n p_0\to \ldots$$

that converges to a **stationary distribution** $p_\infty$. Such vector is clearly invariant under the dynamics:
$$T p_\infty =p_\infty$$
in other words, $p_\infty$ is a (normalized) eigenvector of $T$ with eigenvalue 1.

**Do all Markov Chains have a stationary distribution?** Not necessarily (think of a simple random walk on an infinitely countable set of states).

**If it exists, will $T^n$ converge?** Not necessarily, the process could be periodic.

**If it exists, is the stationary distribution unique?** Not necessarily, there may be many of them. To ascertain uniqueness, we need to look at the spectrum of $T$.

$T$ always has at least one eigenvalue 1. Indeed, we can write the column-normalization condition as
$$\boldsymbol{1}^T T=\boldsymbol{1}^T$$
with $\boldsymbol{1}$ the vector with entries all $1$, implying that $T$ always has a left eigenvector with eigenvalue $1$.

Furthermore, all eigenvalues $\lambda_\alpha$ of $T$ lie in the unit disc in the complex plane $|\lambda_\alpha|\leq1$.

Here's a "proof by example":

In [ ]:
N = 5
num_samples = 100

Ts = torch.rand(num_samples, N, N)
Ts = Ts / torch.sum(Ts, 1, keepdim=True) # using torch here to conveniently compute eigenvalues in parallel for all matrices
eigs = torch.linalg.eigvals(Ts).numpy()

plt.plot(np.real(eigs.flatten()), np.imag(eigs.flatten()), '.', alpha=0.5);
plt.gca().set_aspect('equal')
circle = np.exp(1j*np.linspace(0,2*np.pi,100))
plt.plot(np.real(circle), np.imag(circle));
plt.xlabel('Re λ')
plt.ylabel('Im λ');

If all entries of $T$ are positive, then its largest eigenvalue $\lambda_{\text{max}}=1$ is simple (i.e. its algebraic multiplicity is $1$), its associated eigenvector has positive entries, and all other eigenvalues lie stricly inside the unit disk.

For general non-negative entries, the eigenvalue 1 could be degenerate: in such a case the stationary distribution is not unique and depends on the initial state.

If for any pair of state $x'$ and $x$ there is a path from $x$ to $x'$ with non-zero probability, the Markov process is *irreducible*. In such a case, the process has a unique stationary distribution.
  
In such a case $T$ has only one eigenvalue $1$ with a corresponding eigenvector possessing positive entries. The remaining eigenvalues are either inside the unit disk or have unit norm. By the **Perron-Frobenius** theorem, an irreducible Markov process $T$ of *periodicity $d$* has $d$ (simple) eigenvalues given by
$$\lambda_m=\exp\left(\frac{2 \pi i m}{d}\right), m=0,\ldots,d-1$$

In absence of periodicity, an irreducible chain is **ergodic** - although some authors call all irreducible chains ergodic - and any initial condition converges to the stationary distribution.

### *Bonus: Ergodicity and irreducibility*

More generally, one can define irreducibility for a subset of states $\mathcal{C} \subset \mathcal{S}$ if for any state
$x \in \mathcal{C}$ there is a finite probability to visit any other state $x' \in \mathcal{C}$:
$$x=x^0,x^1,\ldots,x^k=x'$$
with $T(x^i|x^{i-1})\>0,i=1,\ldots,k$.

A subset of states $\mathcal{C} \subset \mathcal{S}$ is called *closed* when
the Markov process can never escape from $\mathcal{C}$, once entered:
$$T(x'|x)=0~~~\mbox{for all}~~x \in \mathcal{C}, x' \notin \mathcal{C}.$$
In general, we can decompose the state space $\mathcal{S}$ uniquely
into closed irreducible subsets $\mathcal{C}_i$
$$\mathcal{S} = \mathcal{T} \cup \mathcal{C}_1 \cup \mathcal{C}_2 \ldots,$$
where $\mathcal{T}$ is a set of *transient states*.

### *Bonus: Characteristic times*

The characteristic time it takes to reach stationarity is determined by the other eigenvalues of $T$.

Let us denote the eigenvalues and left and right eigenvectors of $T$ by
$ \lambda_\alpha, l_\alpha, r_\alpha$ [In our example we would have $\alpha=1,\ldots,2^n$, respectively. Also, the number of eigenvalues of $T$ can be generically less than $2^n$. However, for our purposes we can ignore this case.].

In matrix notation we have [$\dagger$ denotes complex conjugation and transpose]:
$$T r_\alpha=\lambda_\alpha r_\alpha$$
$$l_\alpha^\dagger T =\lambda_\alpha l_\alpha^\dagger$$
Since $T$ is a non-symmetric matrix, the left and right eigenvectors
are different, non-orthogonal and complex valued. The eigenvalues are complex valued.
Under rather general conditions each set of eigenvectors spans a
non-orthogonal basis of $C^{N}$.
These two bases are dual in the sense that:
$$l_\alpha^\dagger r_\beta= \delta_{\alpha\beta}.$$

We can expand $T$ on the basis of its eigenvectors:
$$T=\sum_{\alpha=1}^{N} \lambda_\alpha r_\alpha l_\alpha^\dagger$$

It is easy to check that this satisfies the eigen equations:$$T r_\beta =\sum_\alpha \lambda_\alpha r_\alpha l^\dagger_\alpha r_\beta =\sum_\alpha \lambda_\alpha r_\alpha \delta_{\alpha\beta}=\lambda_\beta r_\beta$$
and similarly $l^\dagger_\beta T=\lambda_\beta l^\dagger_\beta$.

In the ergodic case with periodicity 1:
$$p_t=Tp_{t-1}=\ldots=T^t p_0=\sum_\alpha \lambda_\alpha^t r_\alpha (l_\alpha^\dagger p_0) =p_\infty + \sum_{\alpha >1}\lambda_\alpha^t r_\alpha (l_\alpha^\dagger p_0)$$We can write$$\lambda_\alpha^t = |\lambda_\alpha|^t e^{i\phi_\alpha t} = e^{-t/\tau_\alpha} e^{i\phi_\alpha t}\qquad \tau_\alpha= \frac{-1}{\log| \lambda_\alpha|}$$
The characteristic time to converge to the stationary distribution is determined by the largest $|\lambda_\alpha|$.

With higher periodicity the solution oscillates asymptotically:
$$p_t=\sum_{\alpha, |\lambda_\alpha|=1} \lambda_\alpha^t r_\alpha (l_\alpha^\dagger p_0)+\sum_{\alpha, |\lambda_\alpha|<1} \lambda_\alpha^t r_\alpha (l_\alpha^\dagger p_0)\quad\to\quad \sum_{m=0}^{d-1} e^{2\pi i mt/d} r_m (l^\dagger_m p_0)$$For instance $p_0=\frac{1}{2}(r_0 +r_1)$ and $d=4$:$$p_t(x) = \frac{1}{2} (r_0(x) + i^t r_1(x))$$

### *Bonus: Non-ergodic behavior*

A non-irreducible or non-ergodic
Markov process has more than one eigenvalue 1 and
therefore more than one left and right eigenvector with eigenvalue 1.
Let us denote these eigenvectors by
$l_1,\ldots,l_k$ and $r_1,\ldots,r_k$, respectively. Any linear combination of the
right eigenvectors
$$p_\infty=\sum_{\alpha=1}^k \rho_\alpha r_\alpha$$
is therefore a stationary
distribution, with parameters $\rho_\alpha$ such that $p_\infty(x)\ge 0$ for all
$x$ and proper normalization: $\sum_x p_\infty(x)=1$. Thus, there exists a manifold of
dimension $k-1$ of stationary distributions.

The $k$ left eigenvectors with
eigenvalue 1 encode *invariants* of the dynamics
$$l_\alpha^\dagger p_{t+1}=l_\alpha^\dagger T p_t = l_\alpha^\dagger p_t \qquad \alpha=1,\ldots,k$$
Since $l_1\propto (1,\ldots,1)$ the first invariant simply ensures invariance of normalisation $\sum_x p_{t+1}(x)=\sum_x p_t(x)$.
Thus,
$$l_\alpha^\dagger p_0=l_\alpha^\dagger p_\infty\qquad \alpha=1,\ldots,k$$

The invariants determine uniquely the stationary distribution in terms of the initial conditions.
$$p_\infty=\sum_{\alpha=1}^k \rho_\alpha r_\alpha =\sum_{\alpha=1}^k (l_\alpha^\dagger p_0) r_\alpha$$
because $\rho_\alpha=l_\alpha^\dagger p_\infty=l_\alpha^\dagger p_0$.

Note, that in the ergodic case ($k=1$) the dependence on the initial
state disappears, as it should, since $l_1^\dagger p_0=1$ for any initial
distribution $p_0$.

### Detailed balance

The Markov process $T$ satisfies Detailed Balance (DB) if $$\exists p$$such that
$$T(x|x')p(x')=T(x'|x)p(x)~~\mbox{for all}~~x,x'.$$

The crucial property of DB is that:
$$\text{DB}\implies\text{stationarity}$$
since 
$$\sum_{x'}T\left(x|x'\right)p\left(x'\right)=\sum_{x'}T\left(x'|x\right)p\left(x\right)=p\left(x\right)$$
but in general
$$\text{DB}\nLeftarrow\text{stationarity}$$

# Summary of Markov processes

The Markov process can be analysed in terms of the eigenvalues and
eigenvectors of the transition matrix $T$.

1.  There exists always an eigenvalue $\lambda=1$.
      * If this eigenvalue is non-degenerate, the Markov process is called **ergodic**. The stationary distribution is unique and independent on the initial state.
      * If this eigenvalue is degenerate, the Markov process is called **non-ergodic**. The stationary distribution is not unique and depends on the initial state.
2.  Markov processes can have multiple eigenvalues $e^{2\pi i m/d}, m=0,\ldots, d-1$ and $|\lambda|=1$. In this case it is called periodic with period $d$. In the most common case, the Markov process is non-periodic $d=1$.
3.  Eigenvalues with norm close to 1 ($|\lambda|=1-\epsilon$, $\epsilon$ small) imply long convergence times.

# Some examples

#### Stationary distribution for a random walk on a lattice with reflecting boundaries

In [ ]:
N = 20
num_steps = 1000
num_samples = 5000

x0 = 17
x = np.ones(num_samples) * x0
xs = np.zeros((num_steps, num_samples))
for t in range(num_steps):
    xs[t] = x
    x = x + 2 * np.random.randint(2, size=num_samples) - 1
    x = np.clip(x, 0, N)

In [ ]:
# define transition matrix
T = 0.5 * (np.eye(N+1, k=1) + np.eye(N+1, k=-1))
T[0,0] = 0.5
T[N,N] = 0.5

# compute eigenvalues and eigenvectors (use a function specific for hermitian matrices to directly get sorted eigevalues)
eigs, U = np.linalg.eigh(T)

plt.figure(figsize=(10,4))

plt.subplot(131)
plt.imshow(T, cmap='hot');
plt.title("transition matrix T")

plt.subplot(132)
plt.plot(eigs, '.-')
plt.xlabel('i')
plt.ylabel('eig(T)')

plt.subplot(133)
plt.plot(U[:,-1]);
plt.xlabel('i')
plt.ylabel('eigenvector with eig = 1')

plt.tight_layout();

In [ ]:
# evolve probability of states
p = np.zeros(N+1)
p[x0] = 1
ps = np.zeros((num_steps, N+1))
for t in range(num_steps):
    ps[t] = p
    p = T @ p

# visualize evolution of probability over states
num_to_show = 10
t_to_show = np.logspace(0, np.log10(num_steps), num_to_show, dtype=int) - 1

fig, axes = plt.subplots(num_to_show, figsize=(4, 2 * num_to_show))
for i, t in enumerate(t_to_show):
    axes[i].hist(xs[t], bins=np.arange(N+1+1)-0.5, density=True, alpha=0.5, label='data');
    axes[i].plot(ps[t], '.-', label='theory');
    axes[i].set_title(f't = {t}')
    axes[i].set_xlabel('x')
    axes[i].set_ylabel('p(x)');

plt.tight_layout();

#### The simplest periodic Markov Chain

In this example, state 1 and 2 interchange at each step. Correspondingly, the chain has two eigenvalues on the unit disk (period $d=2$):

In [ ]:
T = np.array([[0,1], [1,0]])
v, U = np.linalg.eig(T)
print("T:")
print(T)
print("eigenvalues:", v)
print("eigenvectors:", U)

#### A slightly more complicate periodic Markov chain: random walk on the hypercube in $n$ dimensions

In [16]:
# construct transition matrix for a random walk on an hypercube
n = 3
N = 2**n
allxs = np.zeros((2**n,n))
for ix, x in enumerate(itertools.product(*[[-1.,1.] for i in range(n)])):
    allxs[ix] = x
Δxs = np.abs(allxs[:,None] - allxs[None]).sum(-1)
T = (Δxs == 2) / n

# compute eigenvalues and eigenvectors (use a function specific for hermitian matrices to directly get sorted eigevalues)
eigs, U = np.linalg.eig(T)
p1 = U[:,np.abs(eigs-1)<1e-10]
pm1 = U[:,np.abs(eigs+1)<1e-10]

In [ ]:
# visualize transition matrix, eigenvalues and eigenvectors
plt.figure(figsize=(10,4))

plt.subplot(131)
plt.imshow(T)
plt.title('T')
plt.title("transition matrix T")

plt.subplot(132)
plt.plot(eigs, '.-')
plt.xlabel('i')
plt.ylabel('eig(T)')

plt.subplot(133)
plt.plot(p1, '.-', label='eigv λ = 1');
plt.plot(pm1,'.-', label='eigv λ = -1');
plt.plot(np.prod(allxs, 1), ':.', c='gray', label="parity")
plt.xlabel('i')
plt.ylabel('eigenvectors of T')
plt.legend()

plt.tight_layout();

In [ ]:
# evolve probability of states
# try starting from a random distribution
num_steps = 100
p = np.random.rand(N)
p /= p.sum()
# start from a uniform distribution
# p = np.ones(N) / N

ps = np.zeros((num_steps, N))
for t in range(num_steps):
    ps[t] = p
    p = T @ p

# visualize last steps
last = 10
fig, axes = plt.subplots(last, figsize=(3, 1 * last))
for i in range(last):
    axes[i].plot(ps[num_steps-i-1]);

plt.tight_layout();

In [ ]:
# construct the random walk by randomly flipping one variable at each step
num_samples = 10

x = 2 * np.random.randint(2, size=(num_samples, n)) - 1

xs = np.zeros((num_steps, num_samples, n))
for t in range(num_steps):
    xs[t] = x
    idxs = np.random.choice(n, size=num_samples)
    x[np.arange(num_samples), idxs] *= -1 

# plot parities of configurations through time
parities = np.prod(xs, -1)
upto = 10
plt.plot(parities[:upto, parities[0]>0], c='black');
plt.plot(parities[:upto, parities[0]<0], c='red');
plt.xlabel('t')
plt.ylabel('parity');

# References

[[1]](https://www.youtube.com/watch?v=5ZLtcTZP2js)